# Problem definition

LightGBM Bayesian Optimization HyperOpt

basé sur les sources suivantes:

Sources:

https://github.com/microsoft/LightGBM/issues/695#issuecomment-315591634

https://machinelearningmastery.com/framework-for-imbalanced-classification-projects/

https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

https://sites.google.com/view/lauraepp/parameters
https://sanchom.wordpress.com/tag/average-precision/

In [1]:
# basics
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# preprocess
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import PowerTransformer

# model
from lightgbm import LGBMClassifier

# imbalanced
from imblearn.pipeline import Pipeline

## hyperopt functions
from hyperopt import fmin, hp, tpe, Trials, space_eval
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
from functools import partial

# evalue
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, average_precision_score, roc_auc_score, fbeta_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, auc, log_loss
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve, roc_curve, plot_roc_curve

# resample
from imblearn.over_sampling import ADASYN, SMOTE
from imblearn.under_sampling import OneSidedSelection, NeighbourhoodCleaningRule, TomekLinks

# turn off warning
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Set model
def instance_model(hyperparameters):
    
    # LightGBM classifier
    if pd.DataFrame(hyperparameters.keys())[0][0] == 'lgbm':
        model = LGBMClassifier(**hyperparameters['lgbm'], 
                            n_jobs = -1,
                            random_state = 42,      
                            objective = "binary", 
                            #categorical_feature = categorical_features_index,
                            n_estimators = 9999999,
                            bagging_freq = 1,       
                            #is_unbalance = True,   
                            learning_rate = 0.01)  
       
        # Resampling
        # ADASYN: Adaptive synthetic sampling
        if hyperparameters['lgbm']['sample'] == 'adasyn':
            undersample = ADASYN(random_state=42)
            if hyperparameters['lgbm']['power'] == True:
                power = PowerTransformer(method='yeo-johnson', standardize=True)
                model = Pipeline([('sampling', undersample), 
                                  ('power', power), ('lgbm', model) ])
            else: 
                model = Pipeline([('sampling', undersample), ('lgbm', model) ])
                
        # SMOTE: Synthetic Minority Oversampling Technique
        if hyperparameters['lgbm']['sample'] == 'smote':
            undersample = SMOTE()
            if hyperparameters['lgbm']['power'] == True:
                power = PowerTransformer(method='yeo-johnson', standardize=True)
                model = Pipeline([('sampling', undersample),
                                  ('power', power), ('lgbm', model) ])
            else: 
                model = Pipeline([('sampling', undersample), ('lgbm', model) ])
                
        # Tomek Links: remover exemplos ambiguos
        elif hyperparameters['lgbm']['sample'] == 'tomek':
            undersample = TomekLinks()
            if hyperparameters['lgbm']['power'] == True:
                power = PowerTransformer(method='yeo-johnson', standardize=True)
                model = Pipeline([('sampling', undersample), 
                                  ('power', power), ('lgbm', model) ])
            else:
                model = Pipeline([('sampling', undersample), ('lgbm', model) ])
                
        # Neighborhood Cleaning Rule for Undersampling: 
        # Condensed Nearest Neighbor + Edited Nearest Neighbors
        elif hyperparameters['lgbm']['sample'] == 'ncr': 
            undersample  = NeighbourhoodCleaningRule(n_neighbors=3,
                                                     threshold_cleaning=0.5)
            if hyperparameters['lgbm']['power'] == True:
                power = PowerTransformer(method='yeo-johnson', standardize=True)
                model = Pipeline([('sampling', undersample),
                                  ('power', power), ('lgbm', model) ])
            else:
                model = Pipeline([('sampling', undersample), ('lgbm', model) ])
                
        # One-Sided Selection : 
        # Tomek Links + Condensed Nearest Neighbor 
        elif hyperparameters['lgbm']['sample'] == 'oss':
            undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
            if hyperparameters['lgbm']['power'] == True:
                power = PowerTransformer(method='yeo-johnson', standardize=True)
                model = Pipeline([('sampling', undersample), 
                                  ('power', power), ('lgbm', model) ])
            else:
                model = Pipeline([('sampling', undersample), 
                                  ('lgbm', model) ])
        ## No resampling
        else:
            if hyperparameters['lgbm']['power'] == True:
                power = PowerTransformer(method='yeo-johnson', standardize=True)
                model = Pipeline([('sampling', None), 
                                  ('power', power), ('lgbm', model) ])
            else:
                model = Pipeline([('sampling', None), ('lgbm', model) ])
     
    return model

def to_minimize(hyperparameters, features, target, fit_params):
    # create an instance of the model 
    model = instance_model(hyperparameters)
    
    # train with cross-validation
    resultado = cross_val_score(estimator = model, 
                                X = features, 
                                y = target, 
                                scoring = "average_precision",
                                cv = cv, 
                                fit_params = fit_params,
                                n_jobs = -1,
                                error_score='raise')
    
    return -resultado.mean()

# function to get the optimization history
def extract_space_eval(hp_space, trial):
    
    ## get results
    desempacota_trial = space_eval(space = hp_space, 
                                   hp_assignment = {k: v[0] for (k, v) in trial['misc']['vals'].items() if len(v) > 0})
    
    return desempacota_trial

def unpack_dictionary(dictionary):
    unpacked = {}
    for (key, value) in dictionary.items():
        if isinstance(value, dict):
            unpacked = {**unpacked, **unpack_dictionary(value)}
        else:
            unpacked[key] = value
            
    return unpacked

# Metrics to evaluate model
def evalue_model(model, y_test, X_test, model_name):
    # default cut off
    threshold = 0.5
    
    # predict
    pred_prob = model.predict_proba(X_test)
    
    # pr curve to best threshold
    precision, recall, thresholds = precision_recall_curve(y_test, pred_prob[:, 1])

    # calcule fscore
    fscore_f2  = ((1+4)*precision*recall)/(4*precision+recall) # f2
    fscore_f1  = (2*precision*recall)/(precision+recall) #f1
    fscore_f05 = (1.25*precision*recall)/(0.25*precision+recall) #f05
    
    # get max
    threshold_f2  = thresholds[np.argmax(fscore_f2)]
    threshold_f1  = thresholds[np.argmax(fscore_f1)]
    threshold_f05 = thresholds[np.argmax(fscore_f05)]
    
    # prob true class
    pred_prob = [predicao[1] for predicao in pred_prob]
    
    # apply threshold 05
    pred_class = [instancia >= threshold_f05 for instancia in pred_prob]
    
    # confusion matrix
    cm = confusion_matrix(y_true = y_test, y_pred = pred_class)
    
    # metrics
    dictionary = {'accuracy': accuracy_score(y_true = y_test,y_pred=pred_class),
                  'F05': fbeta_score(y_true=y_test,y_pred=pred_class,beta=0.5),
                  'F1': fbeta_score(y_true=y_test,y_pred=pred_class,beta=1),
                  'F2': fbeta_score(y_true=y_test,y_pred=pred_class,beta=2),
                  'recall': recall_score(y_true = y_test, y_pred = pred_class),
                  'precision': precision_score(y_true=y_test,y_pred=pred_class),
                  'tn': cm[0][0],
                  'fn': cm[1][0],
                  'tp': cm[1][1],
                  'fp': cm[0][1],
                  'logloss': log_loss(y_test, pred_prob),
                  'threshold_f2': threshold_f2,
                  'threshold_f05': threshold_f05,
                  'threshold_f1': threshold_f1,
                  'auc': roc_auc_score(y_true = y_test, y_score = pred_prob),
                  'average_precision':average_precision_score(y_true=y_test,y_score=pred_prob),
                  'aucpr': auc(recall, precision),
                  'model_name': model_name}
    
    return dictionary

def plot_auc_pr(name, labels, predictions,n=0.5, **kwargs):
  p, r, _ = precision_recall_curve(labels, predictions)

  plt.plot(100*r, 100*p, label=name, linewidth=2, **kwargs)
  plt.xlabel('Recall [%]')
  plt.ylabel('Precision [%]')
  plt.xlim([-0.5,100])
  plt.title('Precision-Recall Curve')
  plt.ylim([20,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('drive/MyDrive/data/P7/clientsdata.csv')



In [ ]:
liste=df.columns
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for  bcl in liste:
    df[bcl] = le.fit_transform(df[bcl].astype(str))

# Modeling

Separate explanatory variables from the target

In [ ]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train = pd.DataFrame(X_train.values, columns=X.columns)
X_test  = pd.DataFrame(X_test.values, columns=X.columns)
y_train = y_train.values
y_test  = y_test.values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print("Train: non_remboursement =", sum(y_train), "remboursement =", len(y_train) - sum(y_train))
print("Val:   non_remboursement =", sum(y_val), " remboursement =", len(y_val) - sum(y_val))
print("Test:  non_remboursement =", sum(y_test), " remboursement =", len(y_test) - sum(y_test))

eval_set = [(pd.DataFrame(X_val), pd.DataFrame(y_val))]

Define cross-validation method as stratified (due to unbalanced database) and shuffle to avoid ordering bias

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Baseline model

In [ ]:
%%time

model_lgbm = LGBMClassifier(n_jobs = -1, random_state = 42, objective = "binary",
                            #categorical_feature= categorical_features_index,
                            n_estimators = 9999999,
                            bagging_freq = 1,
                            #boosting = "dart",
                            learning_rate = 0.01,
                            is_unbalance = True)

fit_params={'lgbm__early_stopping_rounds': 100, 
            'lgbm__eval_metric': 'average_precision',
            'lgbm__verbose': True,
            'lgbm__eval_set': eval_set}

undersample = None

model_lgbm_baseline = Pipeline([('sample', undersample), 
                                ('lgbm', model_lgbm) ])

model_lgbm_baseline_cv = cross_val_score(model_lgbm_baseline, X_train, y_train, 
                                        cv = cv, 
                                        scoring = "average_precision", 
                                        fit_params = fit_params, 
                                        n_jobs=-1, 
                                         error_score='raise')

print("cross-validation Average Precision:",f"{model_lgbm_baseline_cv.mean():.3f} STD:{model_lgbm_baseline_cv.std():.2f}")

## Bayes Optimization 

In [ ]:
scale_pos_weight_max = int((len(y_train) - sum(y_train)) / sum(y_train))
scale_pos_weight_max

In [ ]:
# choices source: https://github.com/microsoft/LightGBM/issues/695#issuecomment-315591634
hp_space_lgbm = {
    'lgbm': {
        # number of trees and learning rate -------------
        #'n_estimators': ho_scope.int(hp.quniform('n_estimators',100,600,100)), # eval autotune
        #'learning_rate': hp.loguniform('learning_rate',np.log(1e-5),np.log(0.05)), # eval autotune
        # tree depth ------------------------------------
        #'max_depth':  ho_scope.int(hp.quniform('max_depth',2,12,1)),
        #'num_leaves': hp.choice(label = 'num_leaves', options = [15, 31, 63, 127, 255, 511, 1023, 2047, 4095]),
        #'min_child_weight':  ho_scope.int(hp.quniform('min_child_weight',0,X_train.shape[0]/100,1)),
        # conservative update step ----------------------
        ##'max_delta_step': ho_scope.int(hp.quniform('max_delta_step',1,10,1)),
        # sampling --------------------------------------
        'subsample': hp.uniform('subsample',0.4,1), 
        'colsample_bytree': hp.uniform('colsample_bytree',0.4,1),
        #'feature_fraction': hp.uniform('feature_fraction',0.2,0.7),
        # regularization --------------------------------
        'reg_lambda': hp.loguniform('reg_lambda',np.log(1e-4),np.log(10)),
        'reg_alpha': hp.loguniform('reg_alpha',np.log(1e-4),np.log(10)),
        ##'min_gain_to_split': hp.loguniform('min_gain_to_split',np.log(1e-4),np.log(2)),
        # specific lgbm ---------------------------------
        #'min_child_samples': ho_scope.int(hp.quniform('min_child_samples',10,500,100)),
        # set weights for balancing ---------------------
        'scale_pos_weight' : ho_scope.int(hp.loguniform('scale_pos_weight',np.log(1),np.log(scale_pos_weight_max))),
        # Dart Booster ----------------------------------
        #'drop_rate': hp.uniform('drop_rate',0,1),
        #'skip_drop': hp.uniform('skip_drop',0,1)
        # Pipeline parameters ---------------------------
        # Sampling
        'sample':  hp.choice(label = 'sample', options = [None, 'tomek', 'ncr','oss', 'smote']),
        # Boxcox
        'power': hp.choice(label = 'power', options = [False, True])
    }
}

In [ ]:
## criando instancia do Trials
interactions_lgbm = Trials()

In [ ]:
fit_params={'lgbm__early_stopping_rounds': 100,
            'lgbm__eval_metric': 'average_precision',
            'lgbm__verbose': False,
            'lgbm__eval_set': eval_set}

In [ ]:
%%time

## run optimization
optimization = fmin(fn = partial(to_minimize, features = X_train, target = y_train, fit_params = fit_params),
                  space = hp_space_lgbm, 
                  algo = tpe.suggest,
                  trials = interactions_lgbm,
                  max_evals = int(2), 
                  rstate = np.random.RandomState(42))

In [ ]:
## save history 
lgbm_history = pd.DataFrame([unpack_dictionary(extract_space_eval(hp_space_lgbm, x)) for x in interactions_lgbm.trials])

In [ ]:
print(lgbm_history)

## Understand train results

In [ ]:
optimization

In [ ]:
sns.scatterplot(x = lgbm_history.index,  data = lgbm_history)
plt.title('Optimization History')
plt.xlabel(xlabel = 'Interaction')
plt.ylabel(ylabel = 'Average Precision')

In [ ]:
sns.countplot(x = 'sample', data = pd.DataFrame(lgbm_history['sample'].apply(lambda x: "None" if x == None else x)))
plt.title('Times each criterion was selected')
plt.xlabel(xlabel = 'sample')
plt.ylabel(ylabel = 'Interactions')

In [ ]:
sns.countplot(x = 'power', data = lgbm_history)
plt.title('Times each criterion was selected')
plt.xlabel(xlabel = 'YeoJhonson')
plt.ylabel(ylabel = 'Interactions')

In [ ]:
selected_hyperparameters = space_eval(space = hp_space_lgbm, hp_assignment = optimization)
selected_hyperparameters

In [ ]:
model_lgbm_bayeshp = instance_model(hyperparameters=selected_hyperparameters)

# Evalue

In [ ]:
%%time
classifiers = {
    "LGBMBaseline": model_lgbm_baseline,
    "LGBMBayesOpt": model_lgbm_bayeshp
}

# df to store metrics 
results_lgbm = pd.DataFrame(columns= ['metric', 'model_name', 'aucpr', 'average_precision', 'auc', 'accuracy', 'F05', 'F1', 'F2', 'recall', 'precision', 'tn', 'fn', 'tp', 'fp', 'logloss', 'threshold_f2', 'threshold_f05', 'threshold_f1'])

# df to save predictions
pred_df = pd.DataFrame(y_test,index=None)

for key, classifier in classifiers.items():
    print("Running", key)
    model          = classifier.fit(X_train, y_train, 
                                    lgbm__early_stopping_rounds= 100, 
                                    lgbm__eval_metric= 'average_precision',
                                    lgbm__verbose = 0,
                                    lgbm__eval_set= eval_set)
    pred_df[key]   = model.predict_proba(X_test)[:,1]
    training_score = evalue_model(model,y_test, X_test, key)
    df             = pd.DataFrame(training_score.items(), columns = ["metric", "value"])
    results_lgbm   = results_lgbm.append(df.set_index('metric').T)

In [ ]:
results_lgbm.drop('metric', axis=1).reset_index().drop('index', axis=1).sort_values("aucpr", ascending = False)

model_name	aucpr	average_precision	auc	accuracy	F05	F1	F2	recall	precision	tn	fn	tp	fp	logloss	threshold_f2	threshold_f05	threshold_f1
1	LGBMBayesOpt	0.249034	0.249211	0.759235	0.901371	0.304126	0.267922	0.23942	0.223565	0.334237	54327	3855	1110	2211	0.245539	0.08685	0.251061	0.15111
0	LGBMBaseline	0.204705	0.193407	0.718382	0.89342	0.249562	0.223065	0.201654	0.189527	0.271025	54007	4024	941	2531	0.27711	0.101407	0.113087	0.107935

AUCPR

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure(figsize=(12,20))
sns.set_style("whitegrid")
plot_auc_pr("LGBMBaseline", y_test, pred_df["LGBMBaseline"], color=colors[0])
plot_auc_pr("LGBMBayesOpt", y_test ,pred_df["LGBMBayesOpt"], color=colors[1],linestyle='--')
plt.legend(loc='lower left')

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()
plt.figure(figsize=(20,20))
ax.barh(X.columns[np.argsort(model_lgbm_bayeshp.steps[1][1].feature_importances_)][::-1], 
        sorted(model_lgbm_bayeshp.steps[1][1].feature_importances_, reverse=True),
        align='center')
ax.set_yticks(X.columns)
ax.invert_yaxis() 
ax.set_xlabel('Importance')

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [ ]:
import lightgbm as lgb
clf1 = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bylevel=0.8,
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=31,
               objective=None, random_state=44, reg_alpha=0.0, reg_lambda=0.0,
               scale_pos_weight=1, silent=False, subsample=0.8,
               subsample_for_bin=200000, subsample_freq=0)  

model=clf1.fit(X,y,sample_weight=None)

In [ ]:
# save the knn_model to disk
import pickle
filename = 'drive/MyDrive/data/P7/clientsdata.pkl'
pickle.dump(clf1, open(filename, 'wb'))

In [ ]:
fpr,tpr,_ = metrics.roc_curve(yTest,yPredLGB)
rocAuc = metrics.auc(fpr, tpr)
plt.figure(figsize=(12,6))
plt.title('ROC Curve')
sns.lineplot(fpr, tpr, label = 'AUC for LightGBM Model = %0.2f' % rocAuc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
!pip install shap

In [ ]:
import shap

classifier_shap = shap.KernelExplainer(sklearn_regressor.predict, X_train)

shap_results = classifier_shap.shap_values(X_test.iloc[0])

shap.waterfall_plot(classifier_shap.expected_value,shap_values,X_test.iloc[0])


Sources:

- <https://github.com/microsoft/LightGBM/issues/695#issuecomment-315591634>
- <https://machinelearningmastery.com/framework-for-imbalanced-classification-projects/>
- <https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/>
- <https://sites.google.com/view/lauraepp/parameters>
- <https://sanchom.wordpress.com/tag/average-precision/>